In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from bs4 import BeautifulSoup
import sys
import os

import sklearn
from sklearn import cross_validation
from sklearn import svm
from sklearn import metrics

reload(sys)
#sys.setdefaultencoding("ISO-8859-1")
sys.setdefaultencoding("UTF-8")

In [2]:
def clean_text(raw_text):
    review_text = BeautifulSoup(raw_text).get_text()
    words = review_text.lower().split()
    return(" ".join(words))

In [3]:
languages = ["english"]
datafiles = ["summary-english-truth.txt"]
tasks = ["age"]
for language, datafile in zip(languages, datafiles):
    train = pd.read_csv(datafile, header=0, delimiter="\t", quoting=1)
    num_text = train["text"].size
    clean_train_data = []
    
    for i in xrange( 0, num_text):
        clean_train_data.append( clean_text( train["text"][i] ) )

        vectorizer = TfidfVectorizer(analyzer = "word", tokenizer = None, preprocessor = None)
        train_x = vectorizer.fit_transform(clean_train_data)
        train_x = train_x.toarray()
    print "shape: ", train_x.shape
    rows, cols = train_x.shape

In [4]:
with open('age-important-words-using-info-gain.txt') as f:
    alist = [line.rstrip() for line in f]
all_indices_ranked = alist[0].split(',')
all_indices_ranked = [int(x) for x in all_indices_ranked]
all_indices_ranked = [x-1 for x in all_indices_ranked]

In [5]:
train = pd.read_csv("summary-english-truth.txt", header=0, delimiter="\t", quoting=1)
train_y = train['age']
list_of_scores = []

In [6]:
degrees = [1,2,3]
C = [10**-4, 10**-3, 10**-1, 1, 10**1, 10**3, 10**4]
C

[0.0001, 0.001, 0.1, 1, 10, 1000, 10000]

In [7]:
num_features = 9000
xx = [all_indices_ranked[x] for x in range(0, num_features)]
xx = tuple(xx)
smaller_train_x = train_x[:, xx]

In [8]:
list_of_scores = []
for degree in degrees:
    for one_C in C:
        clf = svm.SVC(kernel='poly', degree=degree, coef0=one_C, gamma=1)
        scoring_function = 'accuracy'
        scores = cross_validation.cross_val_score(clf, smaller_train_x, train_y, cv=10, scoring=scoring_function)
        list_of_scores.append(scores)

In [9]:
a = [x.mean() for x in list_of_scores]
a

[0.63208333333333333,
 0.63208333333333333,
 0.63208333333333333,
 0.63208333333333333,
 0.63208333333333333,
 0.63208333333333333,
 0.63208333333333333,
 0.61916666666666664,
 0.61916666666666664,
 0.62541666666666662,
 0.71041666666666659,
 0.80249999999999999,
 0.80249999999999999,
 0.80249999999999999,
 0.51208333333333333,
 0.51208333333333333,
 0.59916666666666663,
 0.75,
 0.8091666666666667,
 0.8091666666666667,
 0.8091666666666667]

In [10]:
clf1 = svm.SVC(kernel='poly', degree=1, coef0=1, C=1, gamma=1)
scores1 = cross_validation.cross_val_score(clf1, smaller_train_x, train_y, cv=10, scoring='accuracy')
scores1.mean()

0.63208333333333333

In [11]:
clf2 = svm.SVC(kernel='linear', C=1, coef0=1)
scores2 = cross_validation.cross_val_score(clf2, smaller_train_x, train_y, cv=10, scoring='accuracy')
scores2.mean()

0.63208333333333333

In [12]:
from scipy import stats
import pandas as pd
p_value_matrix = np.zeros((len(list_of_scores), len(list_of_scores)))
i = range(0, len(list_of_scores))
#sig values
for treatment1,x in zip(list_of_scores,i):
    for treatment2,y in zip(list_of_scores,i):
        z_stat, p_val = stats.ranksums(treatment1, treatment2)
        p_value_matrix[x,y] = p_val

In [13]:
list_of_accuracies = [x.mean() for x in list_of_scores]
df = pd.DataFrame(data = p_value_matrix, columns=list_of_accuracies)
df.index = list_of_accuracies

In [14]:
df.to_csv("scrap.csv", sep='\t', encoding='utf-8')

In [15]:
null_disproved = df[df < 0.05]

In [16]:
null_disproved.to_csv("age-poly-scrap.csv", sep='\t', encoding='utf-8')

In [17]:
#rbf experiments

In [18]:
gammas = [1e-4, 1e-3, 1e-2, 1, 1e2, 1e3, 1e4]
C =  [1e-4, 1e-3, 1e-2, 1, 1e2, 1e3, 1e4]
C

[0.0001, 0.001, 0.01, 1, 100.0, 1000.0, 10000.0]

In [19]:
list_of_scores = []
for g in gammas:
    for one_C in C:
        clf = svm.SVC(kernel='rbf', gamma=g, C=one_C)
        scoring_function = 'accuracy'
        scores = cross_validation.cross_val_score(clf, smaller_train_x, train_y, cv=10, scoring=scoring_function)
        list_of_scores.append(scores)

In [20]:
list_of_accuracies = [x.mean() for x in list_of_scores]
list_of_accuracies

[0.39500000000000002,
 0.39500000000000002,
 0.39500000000000002,
 0.39500000000000002,
 0.39500000000000002,
 0.61874999999999991,
 0.69708333333333328,
 0.39500000000000002,
 0.39500000000000002,
 0.39500000000000002,
 0.39500000000000002,
 0.6120833333333332,
 0.69708333333333328,
 0.8091666666666667,
 0.39500000000000002,
 0.39500000000000002,
 0.39500000000000002,
 0.39500000000000002,
 0.6908333333333333,
 0.80249999999999999,
 0.80249999999999999,
 0.39500000000000002,
 0.39500000000000002,
 0.39500000000000002,
 0.65166666666666662,
 0.80249999999999999,
 0.80249999999999999,
 0.80249999999999999,
 0.39500000000000002,
 0.39500000000000002,
 0.39500000000000002,
 0.39500000000000002,
 0.39500000000000002,
 0.39500000000000002,
 0.39500000000000002,
 0.39500000000000002,
 0.39500000000000002,
 0.39500000000000002,
 0.39500000000000002,
 0.39500000000000002,
 0.39500000000000002,
 0.39500000000000002,
 0.39500000000000002,
 0.39500000000000002,
 0.39500000000000002,
 0.3950000000

In [21]:
p_value_matrix = np.zeros((len(list_of_scores), len(list_of_scores)))
i = range(0, len(list_of_scores))
#sig values
for treatment1,x in zip(list_of_scores,i):
    for treatment2,y in zip(list_of_scores,i):
        z_stat, p_val = stats.ranksums(treatment1, treatment2)
        p_value_matrix[x,y] = p_val

In [22]:
df = pd.DataFrame(data = p_value_matrix, columns=list_of_accuracies)
df.index = list_of_accuracies

In [23]:
null_disproved = df[df < 0.05]

In [24]:
null_disproved.to_csv("age-rbf-scrap.csv", sep='\t', encoding='utf-8')

In [25]:
gammas

[0.0001, 0.001, 0.01, 1, 100.0, 1000.0, 10000.0]